## init

In [62]:
# import sys 
# sys.path.append('pyfiles/')

from pyfiles import (parametrics as par,
    complex as cplx,
    matrix as mat,
    vineyard as vin,
    input as inp,
    plot as ourplot
)

# so we don't have to reboot kernel every time we modify classes.py
%load_ext autoreload
%autoreload 2

# math stuff
import numpy as np
import math
from collections import defaultdict
# visualization
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.ndimage import gaussian_filter1d
from scipy.spatial import distance
from scipy.stats import qmc
from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline

import csv
import string

# for polygon grid
from matplotlib.path import Path as mplPath

######## NOTE: from this point on, order is important. 
# it would be good to make the code more robust, but right
# now, there are ordered dependencies between functions.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## this is the new, streamlined thing

In [69]:
our_complex = inp.read_obj("input/two_triangles.obj")
o = cplx.ordering.by_dist_to(our_complex, (0.75, 0.40))
matrix = mat.bdmatrix.from_ordering(o)
with ourplot.PandasMatrix(matrix, o) as p:
    matrix.reduce(every_step=p.every_step)

def every_step(mat, indices, old_j):
    (j, j0) = indices # col(j) ^= col(j0)
    if not mat[j]:
        # print(f'{j0} killed {j}')
        pass

bettis = defaultdict(int)
def after_column_reduced(mat, j):
    if mat.get(j, set()):
        death = o.simplex(j)
        birth = o.simplex(max(mat.get(j, set())))
        bettis[birth.dim()] -= 1
        print(f'col {j}:')
        print(f'  birth={birth}')
        print(f'  death={death}')
    else:
        s = o.simplex(j)
        bettis[s.dim()] += 1
        print(f'col {j}: reduced to zero: {s}')

matrix.reduce(every_step=every_step, 
              after_column_reduced=after_column_reduced);
print(bettis)

# plot the complex with a heatmap from the key point
# ourplot.plot_complex(our_complex)




,0|∅,1|v1,2|v2,3|e1,4|v0,5|e0,6|e2,7|v3,8|v4,9|e3,10|v5,11|e4,12|e5
0|∅,0,1,1,0,1,0,0,1,1,0,1,0,0
1|v1,0,0,0,1,0,1,0,0,0,0,0,0,0
2|v2,0,0,0,1,0,0,1,0,0,0,0,0,0
3|e1,0,0,0,0,0,0,0,0,0,0,0,0,0
4|v0,0,0,0,0,0,1,1,0,0,0,0,0,0
5|e0,0,0,0,0,0,0,0,0,0,0,0,0,0
6|e2,0,0,0,0,0,0,0,0,0,0,0,0,0
7|v3,0,0,0,0,0,0,0,0,0,1,0,0,1
8|v4,0,0,0,0,0,0,0,0,0,1,0,1,0
9|e3,0,0,0,0,0,0,0,0,0,0,0,0,0


col 0: reduced to zero: simplex ∅ bd []
col 1:
  birth=simplex ∅ bd []
  death=simplex v1 bd [-1]
col 2: reduced to zero: simplex v2 bd [-1]
col 3:
  birth=simplex v2 bd [-1]
  death=simplex e1 bd [1, 2]
col 4: reduced to zero: simplex v0 bd [-1]
col 5:
  birth=simplex v0 bd [-1]
  death=simplex e0 bd [0, 1]
col 6: reduced to zero: simplex e2 bd [2, 0]
col 7: reduced to zero: simplex v3 bd [-1]
col 8: reduced to zero: simplex v4 bd [-1]
col 9:
  birth=simplex v4 bd [-1]
  death=simplex e3 bd [3, 4]
col 10: reduced to zero: simplex v5 bd [-1]
col 11:
  birth=simplex v5 bd [-1]
  death=simplex e4 bd [4, 5]
col 12: reduced to zero: simplex e5 bd [5, 3]
defaultdict(<class 'int'>, {-1: 0, 0: 1, 1: 2})
